# Batch process segmentation made with Ilastik

This code performs the identical steps as in notebook 1A, but in a more streamlined way, allowing for batch processing without visual inspection.

The processing functions have been moved to the process_colonies.py script.

In [1]:
#next two lines make sure that Matplotlib plots are shown properly in Jupyter Notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#main data analysis packages
import numpy as np
import seaborn as sns
import pandas as pd
import dask.array as da

#path handling
import pathlib
import h5py

#dask cash
from dask.cache import Cache
cache = Cache(4e9)  # Leverage 4 gigabytes of memory
cache.register()    # Turn cache on globally

import process_colonies as pc

## Set Paths and Settings

In [8]:
#set path to registered file
path_reg_im = pathlib.Path("/Volumes/ScientificData/Users/Giulia(botgiu00)/Collaborations/Ashley/2023-04-11-agar-pad-processed/")

#set path to Ilastik output file
path_seg_im = pathlib.Path("/Volumes/ScientificData/Users/Giulia(botgiu00)/Collaborations/Ashley/2023-04-11-agar-pad-processed/Probabilities_8bit_export/")

#set path to output csv files
path_data_files = pathlib.Path() 

#set filenames
exp_name = "20230411"

#set to true to calculate distance between colony edges, more accurate that center to center distance, but very slow
calc_edge_dist = True

#specify properties to extract 
prop_list = ['label', 
            'area', 'centroid', 
            'axis_major_length', 'axis_minor_length']

#specify processing settings
settings = {
    #specify the order of the strains in the Ilastik layers
    'idx_SA1'   : 0, #SA1 is GFP
    'idx_SA2'   : 1, #SA2 is RFP
    'idx_BG'    : 2,
    'idx_PA'    : 3,
    #specify the segementation processing parameters for pseudomonas
    'sigma'             : 1, # sigma for gaussian filter
    'threshold_PA'      : 0.5, # threshold for segmentation
    'closing_radius_PA' : 5, # radius for closing operation
    'min_cell_area_PA'  : 20, # minimum area for a cell to be considered
    'max_hole_area_PA'  : 100, # maximum area for a hole to be filled
    #specify the segementation processing parameters for staph
    'sigma'             : 1, # sigma for gaussian filter
    'threshold_SA'      : 0.5, # threshold for segmentation
    'closing_radius_SA' : 5, # radius for closing operation
    'min_cell_area_SA'  : 20, # minimum area for a cell to be considered
    'max_hole_area_SA'  : 100, # maximum area for a hole to be filled    
    #store path metadata
    'path_reg_im'       : path_reg_im,
    'path_seg_im'       : path_seg_im,
    'path_data_files'    : path_data_files,
    'exp_name'          : exp_name
}


#load metadata and add to settings
metadata_path = settings['path_data_files'] / f'agarpad_{exp_name}.csv'
pos_metadata = pd.read_csv(metadata_path, index_col=0)
settings['pos_metadata'] = pos_metadata

## Loop positions
First make sure that all positions are found

In [3]:
pos_list = [f.name for f in sorted(path_seg_im.glob('*_Probabilities.h5'))]
for pos in pos_list: print(pos)

20230411_reg_p000-images_Probabilities.h5
20230411_reg_p004-images_Probabilities.h5
20230411_reg_p008-images_Probabilities.h5
20230411_reg_p015-images_Probabilities.h5
20230411_reg_p016-images_Probabilities.h5
20230411_reg_p020-images_Probabilities.h5


now we loop positions, this will take a while

In [ ]:
df_all = []
csv_dir_pos = settings['path_data_files'] / settings['exp_name']
csv_dir_pos.mkdir(exist_ok=True)

for pos in pos_list:
    try:
        #get psotion number
        file_name = pos.split('-images')[0]
        pos_idx = int(file_name.split('_p')[-1])
        
        print(f"Processing position {pos_idx}")
        
        #construct file names
        file_name_im = f"{exp_name}_reg_p{pos_idx:03d}.h5"
        file_name_seg = f"{exp_name}_reg_p{pos_idx:03d}-images_Probabilities.h5"

        #load registered images
        reg_im_file = h5py.File(settings['path_reg_im']/file_name_im, 'r') #open 
        chunk_size = (1, *reg_im_file['images'].shape[-3:])
        reg_im = da.from_array(reg_im_file['images'], chunks=chunk_size)

        #load segmented images
        seg_im_file = h5py.File(settings['path_seg_im']/file_name_seg, 'r') #open 
        chunk_size = (1, 1,*reg_im_file['images'].shape[-2:])
        seg_prob = da.from_array(seg_im_file['exported_data'], chunks=chunk_size)

        #crop to max frame
        pdata = settings['pos_metadata']
        max_frm = int(pdata.loc[f"pos{pos_idx:03d}","max_frame"]) if f"pos{pos_idx:03d}" in pdata.index else reg_im.shape[0]
        reg_im = reg_im[:max_frm]       
        seg_prob = seg_prob[:max_frm]       

        condition = pdata.loc[f"pos{pos_idx:03d}","condition"] if f"pos{pos_idx:03d}" in pdata.index else ''
        
        #convert to labels
        SA1_labels = pc.process_seg(seg_prob[:,settings['idx_SA1'],:,:], 
                                    sigma = settings['sigma'],
                                    threshold = settings['threshold_SA'],
                                    closing_radius = settings['closing_radius_SA'],
                                    min_cell_area = settings['min_cell_area_SA'],
                                    max_hole_area = settings['max_hole_area_SA'])
                                    
        SA2_labels = pc.process_seg(seg_prob[:,settings['idx_SA2'],:,:], 
                                sigma = settings['sigma'],
                                threshold = settings['threshold_SA'],
                                closing_radius = settings['closing_radius_SA'],
                                min_cell_area = settings['min_cell_area_SA'],
                                max_hole_area = settings['max_hole_area_SA'])                       

        PA_labels = pc.process_seg(seg_prob[:,settings['idx_PA'],:,:], 
                                sigma = settings['sigma'],
                                threshold = settings['threshold_PA'],
                                closing_radius = settings['closing_radius_PA'],
                                min_cell_area = settings['min_cell_area_PA'],
                                max_hole_area = settings['max_hole_area_PA'])      
        
        #track colonies and extract properties
        df_SA1 = pc.track_extract_prop(SA1_labels, prop_list, 
                                 metadata = {'pos': pos_idx, 'strain':'SA1','condition':condition})
        
        df_SA2 = pc.track_extract_prop(SA2_labels, prop_list, 
                                 metadata = {'pos': pos_idx, 'strain':'SA2','condition':condition})
        
        df_PA = pc.track_extract_prop(PA_labels, prop_list, 
                                 metadata = {'pos': pos_idx, 'strain':'PA','condition':condition})
        
        #combine dataframes
        df = pd.concat([df_SA1, df_SA2, df_PA]).reset_index(drop=True)
        
        #add spatial properties
        df = pc.add_centrod_distance(df)
        if calc_edge_dist:
            df = pc.add_edge2edge_distance(df,PA_labels,SA1_labels,SA2_labels)
        
        expname = settings["exp_name"]
        csv_name = csv_dir_pos / f"{expname}_pos{pos_idx:03d}.csv"
        df.to_csv(csv_name)
        df_all.append(df)
        
    except:
         print("Error processing position {}".format(pos_idx))  
        
df_combined = pd.concat(df_all).reset_index(drop=True)  
csv_name = settings['path_data_files'] / f"{expname}_all_data.csv"
df_combined.to_csv(csv_name)
